In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import warnings

# Ignore warnings for clean output
warnings.filterwarnings('ignore')

# Matplotlib settings
%matplotlib inline
sns.set(style="whitegrid")


Loading Raw data ```data.csv``` data

In [3]:
df = pd.read_csv('data/raw/data.csv')
df.head()

,Patient ID,Age,Gender,Blood Pressure,Heart Rate,Temperature,Oxygen Saturation,Respiratory Rate,Symptoms,Medical History,Diagnosis,Admission Date
0,1,82,Male,127/80,56.0,33.6,100,20,Headache,Stroke,Hypertension,2023-05-06
1,2,58,Other,121/83,NaN,37.5,99,12,Fatigue,NaN,Healthy,2023-01-04
2,3,59,Other,121/85,78.0,36.5,90,15,Cough,COVID-19,Asthma,2023-10-10
3,4,21,Female,145/96,89.0,37.0,97,20,Chest Pain,Hypertension,Stroke,2023-05-17
4,5,30,Male,128/78,69.0,37.4,90,15,Chest Pain,COVID-19,Hypertension,2023-02-03
